In [1]:
import pandas as pd
import openai
import json
import time
import matplotlib.pyplot as plt
from tqdm import tqdm  # Import tqdm for the progress bar
from sklearn.metrics import accuracy_score, classification_report, balanced_accuracy_score
from sklearn.metrics import balanced_accuracy_score
import numpy as np

In [2]:
#pip install openai

In [3]:
from openai import OpenAI

In [4]:
openai.__version__


'1.1.1'

### Read in annotated data

In [5]:
df = pd.read_csv("../data/data_splits_stratified/6-2-2_all_classes_enriched_with_kw/test.csv", index_col=0)

In [6]:
df.shape

(534, 8)

In [7]:
df.head()

,pmid,journal_name,title,abstract,keywords,accepted_label,multi_label,binary_label
idx,,,,,,,,
1,12047012,Schizophrenia bulletin,Can clinical practice guide a research agenda?,Articles from this issue of the Bulletin indic...,NaN,Non-systematic-review,1,0
2,28832188,Future medicinal chemistry,Dual/multitargeted xanthone derivatives for Al...,"To date, the current therapy for Alzheimer's d...",Alzheimer's disease| Aβ aggregation| anticholi...,Non-systematic-review,1,0
3,17678496,Expert review of neurotherapeutics,Benefits of occupational therapy in stroke reh...,Stroke is the largest single cause of severe p...,NaN,Non-systematic-review,1,0
4,25649308,Annals of the New York Academy of Sciences,The promise of ketamine for treatment-resistan...,Major depressive disorder (MDD) is one of the ...,antidepressant| bipolar disorder| ketamine| ma...,Non-systematic-review,1,0
5,6312596,La semaine des hopitaux : organe fonde par l'A...,[Clinical and pathogenic aspects of secondary ...,The secondary hyperlipoproteinemias are freque...,NaN,Non-systematic-review,1,0


In [8]:
# Combine the columns
# Implementing custom tags for the combination of journal name, title, and abstract
df['input_journal_title_abstract'] = '<journal>' + df['journal_name'] + '</journal>' + \
                                     '<title>' + df['title'] + '</title>' + \
                                     '<abstract>' + df['abstract'] + '</abstract>'

# Implementing custom tags for the combination of title and abstract only
df['input_title_abstract'] = '<title>' + df['title'] + '</title>' + \
                             '<abstract>' + df['abstract'] + '</abstract>'

In [9]:
df.head(5)

,pmid,journal_name,title,abstract,keywords,accepted_label,multi_label,binary_label,input_journal_title_abstract,input_title_abstract
idx,,,,,,,,,,
1,12047012,Schizophrenia bulletin,Can clinical practice guide a research agenda?,Articles from this issue of the Bulletin indic...,NaN,Non-systematic-review,1,0,<journal>Schizophrenia bulletin</journal><titl...,<title>Can clinical practice guide a research ...
2,28832188,Future medicinal chemistry,Dual/multitargeted xanthone derivatives for Al...,"To date, the current therapy for Alzheimer's d...",Alzheimer's disease| Aβ aggregation| anticholi...,Non-systematic-review,1,0,<journal>Future medicinal chemistry</journal><...,<title>Dual/multitargeted xanthone derivatives...
3,17678496,Expert review of neurotherapeutics,Benefits of occupational therapy in stroke reh...,Stroke is the largest single cause of severe p...,NaN,Non-systematic-review,1,0,<journal>Expert review of neurotherapeutics</j...,<title>Benefits of occupational therapy in str...
4,25649308,Annals of the New York Academy of Sciences,The promise of ketamine for treatment-resistan...,Major depressive disorder (MDD) is one of the ...,antidepressant| bipolar disorder| ketamine| ma...,Non-systematic-review,1,0,<journal>Annals of the New York Academy of Sci...,<title>The promise of ketamine for treatment-r...
5,6312596,La semaine des hopitaux : organe fonde par l'A...,[Clinical and pathogenic aspects of secondary ...,The secondary hyperlipoproteinemias are freque...,NaN,Non-systematic-review,1,0,<journal>La semaine des hopitaux : organe fond...,<title>[Clinical and pathogenic aspects of sec...


### Load key for the OpenAI API 

In [10]:
def load_pass(file_path, key_to_find):
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split("=")
            if len(parts) == 2 and parts[0] == key_to_find:
                found_password = parts[1]
                break
    if found_password:
        print("Found password.")
        return found_password
    else:
        print("Password not found for key:", key_to_find)

Note: You need to create a credentials.txt file with the following content:  
OPENAI=sk-77QXXXXXXXXXXXXXXXXXXXXXXXXXXX  
replace the value after the = sign with your API key.  
Make sure the credentials.txt is added to .gitignore, you don't want to put your password on Git!

In [11]:
openai.api_key = load_pass("./credentials.txt", "OPENAI")


Found password.


In [12]:
client = OpenAI(api_key=openai.api_key)

### Query GPT

To change the task the model is solving, you need to change the text of the prompt and the content text of the system role.  
To change the GPT model used, you need to change the text of the model name when initiating the openai API.  
The function gets as input the input_raw_text, that will be text for information extraction or classification.

In [13]:
import time

DEFAULT_TEMPERATURE = 0
DEFAULT_MAX_TOKENS = 500
DEFAULT_MODEL = "gpt-3.5-turbo"

def query_gpt(input_raw_text, prompt_text, gpt_model="gpt-3.5-turbo", temperature=0, max_retries=5, retry_delay=3):
    # CHANGE gpt_model to the desired model name, see https://platform.openai.com/docs/models (gpt-3.5-turbo and gpt-4-turbo-preview)
    
    # Add a delay at the beginning of the function to avoid overloading the API if there are multiple calls
    # time.sleep(10)  

    system_msg = f"""
    You are an expert assistant specialized in text classification of PubMed abstracts. """

    retries = 0
    while retries < max_retries:
        print("Trying to call OpenAI API...")
        try:
            completion = client.chat.completions.create(
                model=gpt_model,  
                response_format={"type": "json_object"},
                temperature=temperature,
                #max_tokens=2000,
                messages=[
                    {"role": "system", "content": system_msg},
                    {"role": "user", "content": prompt_text + input_raw_text}
                ]
            )
            return completion.choices[0].message.content
        except Exception as e:
            # Handle API error, e.g., retry or log
            print(f"OpenAI API returned an error: {e}")
            time.sleep(retry_delay)  # Wait before retrying
            retries += 1

    raise RuntimeError("Max retries reached. Unable to complete the API call.")


In [14]:
# Define a function to apply GPT queries with a progress bar
def apply_gpt_with_progress(data_series, prompt_text, model="gpt-3.5-turbo"):
    results = []
    total_items = len(data_series)
    # Create a tqdm progress bar
    with tqdm(total=total_items, desc=f"Processing dataset") as pbar:
        for text in data_series:
            result = query_gpt(text, prompt_text, model)
            results.append(result)
            pbar.update(1)  # Update the progress bar

    return results

# Hierarchical: Read binary and further classify

In [15]:
# Parameters to select Binary Predictions file
prompt_ids_to_eval = ["P2", "P3"] #, "P2", "P3", "P4"
model = "gpt-3.5-turbo"
file_to_save_suffix = "hierarchical"

# The predictions based on which prompt to use
prompt_id_to_use_for_binary = "P3"

In [16]:
# Create the list of prediction columns based on the prompt IDs
prediction_columns = [f'gpt_predictions_{prompt_id_to_use_for_binary}']
# Specify the basic columns to include in the DataFrame
basic_columns = ['pmid', 'input_journal_title_abstract', 'accepted_label', 'multi_label', 'binary_label']
# Combine basic columns with the dynamically generated prediction columns
columns_to_read = basic_columns + prediction_columns

In [17]:
target_file_for_eval = f"predictions/{model}_enriched_kw_test_outputs_{'_'.join(prompt_ids_to_eval)}_binary.csv"
df_binary = pd.read_csv(target_file_for_eval)[columns_to_read]
col_name_binary = f'{prediction_columns[0]}_binary'
df_binary.rename(columns={prediction_columns[0]: col_name_binary}, inplace=True)

In [18]:
df_binary.head()

,pmid,input_journal_title_abstract,accepted_label,multi_label,binary_label,gpt_predictions_P3_binary
0,12047012,<journal>Schizophrenia bulletin</journal><titl...,Non-systematic-review,1,0,OTHER
1,28832188,<journal>Future medicinal chemistry</journal><...,Non-systematic-review,1,0,OTHER
2,17678496,<journal>Expert review of neurotherapeutics</j...,Non-systematic-review,1,0,OTHER
3,25649308,<journal>Annals of the New York Academy of Sci...,Non-systematic-review,1,0,OTHER
4,6312596,<journal>La semaine des hopitaux : organe fond...,Non-systematic-review,1,0,OTHER


In [19]:
prediction_columns

['gpt_predictions_P3']

## Further classify studies depending on binary

In [20]:
df_animal = df_binary[df_binary[col_name_binary] == 'ANIMAL'] 
df_animal.head()

,pmid,input_journal_title_abstract,accepted_label,multi_label,binary_label,gpt_predictions_P3_binary
21,32147509,<journal>Neuroscience</journal><title>Neurobio...,Non-systematic-review,1,0,ANIMAL
33,16312938,<journal>Zhongguo zhen jiu = Chinese acupunctu...,Non-systematic-review,1,0,ANIMAL
71,23811310,<journal>Biochemical pharmacology</journal><ti...,Non-systematic-review,1,0,ANIMAL
100,27534431,<journal>Amyotrophic lateral sclerosis & front...,Remaining,0,0,ANIMAL
126,8291133,<journal>Ukrainskii biokhimicheskii zhurnal (1...,Remaining,0,0,ANIMAL


In [21]:
df_other = df_binary[df_binary[col_name_binary] == 'OTHER'] 
df_other.head()

,pmid,input_journal_title_abstract,accepted_label,multi_label,binary_label,gpt_predictions_P3_binary
0,12047012,<journal>Schizophrenia bulletin</journal><titl...,Non-systematic-review,1,0,OTHER
1,28832188,<journal>Future medicinal chemistry</journal><...,Non-systematic-review,1,0,OTHER
2,17678496,<journal>Expert review of neurotherapeutics</j...,Non-systematic-review,1,0,OTHER
3,25649308,<journal>Annals of the New York Academy of Sci...,Non-systematic-review,1,0,OTHER
4,6312596,<journal>La semaine des hopitaux : organe fond...,Non-systematic-review,1,0,OTHER


In [22]:
json_file_path = "./prompt_strategies_hierarchical_shirin.json"
# Load the JSON file
with open(json_file_path, 'r') as file:
    prompts_data = json.load(file)

In [23]:
# Add the IDs of Prompts that you want to test
prompt_ids_to_test = ["P1_HIERARCHY", "P2_HIERARCHY"] #, "P2", "P3", "P4"
model = "gpt-3.5-turbo"
file_to_save_suffix = "hierarchical"

# PROCESS ANIMAL

for prompt in prompts_data["prompts"]:
    prompt_id = prompt["id"]
    prompt_text = prompt["text_animal"]

    if prompt_id in prompt_ids_to_test:
        # Apply GPT predictions
        df_animal[f'gpt_predictions_{prompt_id}_raw'] = apply_gpt_with_progress(df_animal['input_journal_title_abstract'], prompt_text, model)
        ## the below includes error handling in case the json formatting did not work as expected
        df_animal[f'gpt_predictions_{prompt_id}'] = df_animal[f'gpt_predictions_{prompt_id}_raw'].apply(
            lambda x: json.loads(x)['gpt_label'] if isinstance(x, str) and 'gpt_label' in json.loads(x) else x
        )
        df.to_csv(f"predictions/{model}_enriched_kw_test_outputs_{prompt_id_to_use_for_binary}_{'_'.join(prompt_ids_to_test)}_{file_to_save_suffix}.csv") # saving after each prompt strategy has been run, in order not to loose information in case a later strategy fails
    else:
        print(f"Skipping prompt {prompt_id}")

# PROCESS OTHER

for prompt in prompts_data["prompts"]:
    prompt_id = prompt["id"]
    prompt_text = prompt["text_other"]

    if prompt_id in prompt_ids_to_test:
        # Apply GPT predictions
        df_other[f'gpt_predictions_{prompt_id}_raw'] = apply_gpt_with_progress(df_other['input_journal_title_abstract'], prompt_text, model)
        ## the below includes error handling in case the json formatting did not work as expected
        df_other[f'gpt_predictions_{prompt_id}'] = df_other[f'gpt_predictions_{prompt_id}_raw'].apply(
            lambda x: json.loads(x)['gpt_label'] if isinstance(x, str) and 'gpt_label' in json.loads(x) else x
        )
        df.to_csv(f"predictions/{model}_enriched_kw_test_outputs_{prompt_id_to_use_for_binary}_{'_'.join(prompt_ids_to_test)}_{file_to_save_suffix}.csv") # saving after each prompt strategy has been run, in order not to loose information in case a later strategy fails
    else:
        print(f"Skipping prompt {prompt_id}")

Processing dataset:   0%|                                | 0/84 [00:00<?, ?it/s]

Trying to call OpenAI API...


Processing dataset:   1%|▎                       | 1/84 [00:00<01:14,  1.11it/s]

Trying to call OpenAI API...


Processing dataset:   2%|▌                       | 2/84 [00:01<01:10,  1.17it/s]

...

Processing dataset: 100%|████████████████████▉| 448/450 [07:30<00:01,  1.03it/s]

Trying to call OpenAI API...


Processing dataset: 100%|████████████████████▉| 449/450 [07:31<00:00,  1.08it/s]

Trying to call OpenAI API...


Processing dataset: 100%|█████████████████████| 450/450 [07:32<00:00,  1.00s/it]
/var/folders/9j/9lnwvx7s27531pb469pbr08w0000gq/T/ipykernel_52248/132601967.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_other[f'gpt_predictions_{prompt_id}_raw'] = apply_gpt_with_progress(df_other['input_journal_title_abstract'], prompt_text, model)
/var/folders/9j/9lnwvx7s27531pb469pbr08w0000gq/T/ipykernel_52248/132601967.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_other[f'gpt_predictions_{prompt_id}'] =

In [24]:
df_animal.head()

,pmid,input_journal_title_abstract,accepted_label,multi_label,binary_label,gpt_predictions_P3_binary,gpt_predictions_P1_HIERARCHY_raw,gpt_predictions_P1_HIERARCHY,gpt_predictions_P2_HIERARCHY_raw,gpt_predictions_P2_HIERARCHY
21,32147509,<journal>Neuroscience</journal><title>Neurobio...,Non-systematic-review,1,0,ANIMAL,"{\n ""gpt_label"": ""Animal-systematic-review""\n}",Animal-systematic-review,"{\n ""gpt_label"": ""Animal-systematic-review""\n}",Animal-systematic-review
33,16312938,<journal>Zhongguo zhen jiu = Chinese acupunctu...,Non-systematic-review,1,0,ANIMAL,"{\n ""gpt_label"": ""Animal-other""\n}",Animal-other,"{\n ""gpt_label"": ""Animal-other""\n}",Animal-other
71,23811310,<journal>Biochemical pharmacology</journal><ti...,Non-systematic-review,1,0,ANIMAL,"{\n ""gpt_label"": ""Animal-other""\n}",Animal-other,"{\n ""gpt_label"": ""Animal-other""\n}",Animal-other
100,27534431,<journal>Amyotrophic lateral sclerosis & front...,Remaining,0,0,ANIMAL,"{\n ""gpt_label"": ""Animal-other""\n}",Animal-other,"{\n ""gpt_label"": ""Animal-other""\n}",Animal-other
126,8291133,<journal>Ukrainskii biokhimicheskii zhurnal (1...,Remaining,0,0,ANIMAL,"{\n ""gpt_label"": ""Animal-other""\n}",Animal-other,"{\n ""gpt_label"": ""Animal-other""\n}",Animal-other


In [25]:
df_other.head()

,pmid,input_journal_title_abstract,accepted_label,multi_label,binary_label,gpt_predictions_P3_binary,gpt_predictions_P1_HIERARCHY_raw,gpt_predictions_P1_HIERARCHY,gpt_predictions_P2_HIERARCHY_raw,gpt_predictions_P2_HIERARCHY
0,12047012,<journal>Schizophrenia bulletin</journal><titl...,Non-systematic-review,1,0,OTHER,"{\n ""gpt_label"": ""Non-systematic-review""\n}",Non-systematic-review,"{\n ""gpt_label"": ""Non-systematic-review""\n}",Non-systematic-review
1,28832188,<journal>Future medicinal chemistry</journal><...,Non-systematic-review,1,0,OTHER,"{\n ""gpt_label"": ""Human-systematic-review""\n}",Human-systematic-review,"{\n ""gpt_label"": ""Remaining""\n}",Remaining
2,17678496,<journal>Expert review of neurotherapeutics</j...,Non-systematic-review,1,0,OTHER,"{\n ""gpt_label"": ""Human-systematic-review""\n}",Human-systematic-review,"{\n ""gpt_label"": ""Human-non-RCT-non-drug-in...",Human-non-RCT-non-drug-intervention
3,25649308,<journal>Annals of the New York Academy of Sci...,Non-systematic-review,1,0,OTHER,"{\n ""gpt_label"": ""Human-systematic-review""\n}",Human-systematic-review,"{\n ""gpt_label"": ""Human-systematic-review""\n}",Human-systematic-review
4,6312596,<journal>La semaine des hopitaux : organe fond...,Non-systematic-review,1,0,OTHER,"{\n ""gpt_label"": ""Human-systematic-review""\n}",Human-systematic-review,"{\n ""gpt_label"": ""Remaining""\n}",Remaining


In [26]:
appended_df = pd.concat([df_animal, df_other], ignore_index=True)
appended_df.shape

(534, 10)

In [27]:
appended_df.head()

,pmid,input_journal_title_abstract,accepted_label,multi_label,binary_label,gpt_predictions_P3_binary,gpt_predictions_P1_HIERARCHY_raw,gpt_predictions_P1_HIERARCHY,gpt_predictions_P2_HIERARCHY_raw,gpt_predictions_P2_HIERARCHY
0,32147509,<journal>Neuroscience</journal><title>Neurobio...,Non-systematic-review,1,0,ANIMAL,"{\n ""gpt_label"": ""Animal-systematic-review""\n}",Animal-systematic-review,"{\n ""gpt_label"": ""Animal-systematic-review""\n}",Animal-systematic-review
1,16312938,<journal>Zhongguo zhen jiu = Chinese acupunctu...,Non-systematic-review,1,0,ANIMAL,"{\n ""gpt_label"": ""Animal-other""\n}",Animal-other,"{\n ""gpt_label"": ""Animal-other""\n}",Animal-other
2,23811310,<journal>Biochemical pharmacology</journal><ti...,Non-systematic-review,1,0,ANIMAL,"{\n ""gpt_label"": ""Animal-other""\n}",Animal-other,"{\n ""gpt_label"": ""Animal-other""\n}",Animal-other
3,27534431,<journal>Amyotrophic lateral sclerosis & front...,Remaining,0,0,ANIMAL,"{\n ""gpt_label"": ""Animal-other""\n}",Animal-other,"{\n ""gpt_label"": ""Animal-other""\n}",Animal-other
4,8291133,<journal>Ukrainskii biokhimicheskii zhurnal (1...,Remaining,0,0,ANIMAL,"{\n ""gpt_label"": ""Animal-other""\n}",Animal-other,"{\n ""gpt_label"": ""Animal-other""\n}",Animal-other


In [28]:
appended_df.to_csv(f"predictions/{model}_enriched_kw_test_outputs_{prompt_id_to_use_for_binary}_{'_'.join(prompt_ids_to_test)}_{file_to_save_suffix}.csv") # saving after each prompt strategy has been run, in order not to loose information in case a later strategy fails